In [4]:
import PIL

In [7]:
import requests
import gradio as gr
from PIL import Image, ImageDraw


def request_object_detection(image_path):

    endpoint = "https://fimtrus-computer-vision.cognitiveservices.azure.com/computervision/imageanalysis:analyze?model-name=detectionfork&language=en&gender-neutral-caption=false&api-version=2023-04-01-preview"
    api_key = "4e650db659a84880ad9bff6b57af6cf2"

    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }

    with open(image_path, "rb") as image:
        image_data = image.read()

    response = requests.post(endpoint,
                            headers=headers,
                            data=image_data)

    if response.status_code == 200:
        response_json = response.json()
        object_list = response_json["customModelResult"]["objectsResult"]["values"]
        return object_list
    else:
        return None

def draw(image_path, object_list):

    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    for c in object_list:
        bounding_box = c['boundingBox']
        if len(c['tags']) > 0:
            name = c['tags'][0]['name']
            confidence = c['tags'][0]['confidence']
        else:
            name = ''
            confidence = ''

        x = bounding_box['x']
        y = bounding_box['y']
        w = bounding_box['w']
        h = bounding_box['h']

        start_point = (x, y)
        end_point = (x + w, y + h)

        import numpy as np

        random_colors = tuple(np.random.randint(0, 255, 3))

        draw.rectangle([start_point, end_point], outline=random_colors, width=2)
        draw.text((x+5, y+5), "{}({:.2f}%)".format(name, confidence * 100), fill=random_colors)

        #print("Detect Object: ", name, confidence, x, y, w, h)
    return image


def change_input_image(input_image):
    object_list = request_object_detection(input_image)
    draw_image = draw(input_image, object_list)
    print(object_list)
    return draw_image


with gr.Blocks() as demo:
    input_image = gr.Image(label="이미지 선택", type="filepath")
    result_image = gr.Image(label="결과 이미지")

    input_image.change(fn=change_input_image, inputs=[input_image], outputs=[result_image])

demo.launch()

#file_path = "./fork_5.jpg"
#request_object_detection(file_path)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


[{'id': '1', 'boundingBox': {'x': 132, 'y': 191, 'w': 631, 'h': 244}, 'tags': [{'name': 'fork', 'confidence': 0.9326171875}]}]
[{'id': '1', 'boundingBox': {'x': 99, 'y': 215, 'w': 663, 'h': 105}, 'tags': [{'name': 'fork', 'confidence': 0.916015625}]}, {'id': '2', 'boundingBox': {'x': 98, 'y': 166, 'w': 659, 'h': 139}, 'tags': [{'name': 'fork', 'confidence': 0.12939453125}]}]
[{'id': '1', 'boundingBox': {'x': 195, 'y': 277, 'w': 485, 'h': 210}, 'tags': [{'name': 'fork', 'confidence': 0.88720703125}]}]


In [ ]:
"""
{
    "customModelResult": {
        "objectsResult": {
            "values": [
                {
                    "id": "1",
                    "boundingBox": {
                        "x": 99,
                        "y": 215,
                        "w": 663,
                        "h": 105
                    },
                    "tags": [
                        {
                            "name": "fork",
                            "confidence": 0.916015625
                        }
                    ]
                },
                {
                    "id": "2",
                    "boundingBox": {
                        "x": 98,
                        "y": 166,
                        "w": 659,
                        "h": 139
                    },
                    "tags": [
                        {
                            "name": "fork",
                            "confidence": 0.12939453125
                        }
                    ]
                }
            ]
        }
    },
    "modelVersion": "2023-02-01-preview",
    "metadata": {
        "width": 816,
        "height": 612
    }
}
"""